# Introduction to Gurobi and Gurobipy
The purpose of this exercise is to introduce you to mathematical optimization problems using the python API gurobipy. <br>
- Section 1 presents how to install gurobi and necessary licenses and dependencies
- Section 2 introduces a simple example and exercise to familiarize yourselves with gurobi basics. 
- Section 3 introduces a more general way to formulate and solve optimization problems. 
- Section 4 gives a (very) basic introduction to object-oriented programming, and exemplifies how object-oriented programming can be used to structure optimization problems in power systems. <br>
<b>We recommend using a similar structure for the group assignments.<b>

## Section 1. Installation Guide

### What is Gurobi/Gurobipy?

- Gurobi is a mathematical solver, combining advanced algorithms to solve numerically complex mathwematical optimization problems.
- Gurobipy is a python API used to formulate and solve mathematical optimization problems in python (similar to pyomo or JuMP in Julia).
- It is developed by the same company which develops the gurobi solver. 


### Installing Gurobipy


You can install gurobipy by running:

In [28]:
#pip install gurobipy
# Comment this line after installing gurobipy

Everytime you use Gurobi, you will need to import the package ```gurobipy```. The specific module ```GRB``` is commonly imported separately, as it is used frequently. 

In [29]:
# Import packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np 

### Free Academic LIcense

You then need to obtain a free Academic Named-User License, following these steps:

 1) Register for a free [Gurobi account as an academic and log in](https://portal.gurobi.com/iam/register/?_gl=1*ah7zi4*_up*MQ..*_gs*MQ..&gclid=CjwKCAjwlaTGBhANEiwAoRgXBb0o3PUl8z1tzZOZ9p3KbQPezzjJDyr4wWWdA-fs1K6uV5dppoNYihoCd98QAvD_BwE)
2) Visit the [Download Gurobi Optimizer page](https://www.gurobi.com/downloads/gurobi-software/?_gl=1*ah7zi4*_up*MQ..*_gs*MQ..&gclid=CjwKCAjwlaTGBhANEiwAoRgXBb0o3PUl8z1tzZOZ9p3KbQPezzjJDyr4wWWdA-fs1K6uV5dppoNYihoCd98QAvD_BwE) and download the version you need, as well as the README.txt.
3) Follow the instructions in README.txt to install the software.
4) Once installed, visit the [Gurobi User Portal]() to request your free **Named-User** License.
5) Next, run grbgetkey using the argument provided on the Academic License Detail page (ex: grbgetkey ae36ac20-16e6-acd2-f242-4da6e765fa0a). The grbgetkey program will prompt you to store the license file on your machine.

 **Note that you must be connected to DTU network or eduroam when downloading the academic license for the first time.**

If you encounter an “ERROR 303” message when running grbgetkey, please see the article, [How do I resolve an “ERROR 303” from grbgetkey?](https://support.gurobi.com/hc/en-us/articles/360038994471-How-do-I-resolve-an-ERROR-303-when-running-grbgetkey?_gl=1%2A2dyq1p%2A_up%2AMQ..%2A_gs%2AMQ..&gclid=CjwKCAjwlaTGBhANEiwAoRgXBb0o3PUl8z1tzZOZ9p3KbQPezzjJDyr4wWWdA-fs1K6uV5dppoNYihoCd98QAvD_BwE).

Your license will be valid for up to one year. You can request additional Academic Named-User licenses via the User Portal as long as you maintain eligibility.

This step-by-step video provides a detailed overview of the installation process: 

[![Watch on YouTube](https://img.youtube.com/vi/fRKhao2bzsY/hqdefault.jpg)](https://www.youtube.com/watch?v=fRKhao2bzsY)



## Section 2. Getting Started with Optimization in Gurobipy

### Simple example 

Let's use the following problem as an example:

$$
  \begin{align}
      \textrm{minimize} \quad &30x_1 + 20x_2 \\
      \textrm{subject to} \quad &0.6x_1 + 0.2x_2 \geq 60 \\
      &0.4x_1 + 0.8x_2 \geq 100 \\
      &x_1 \geq 0, x_2 \geq 0 \\
  \end{align}
$$

#### Initialize model

We initialize a model object in which we'll store the problem.

In [30]:
#create and name a new gurobi model
model = gp.Model("My_LP_problem")

#### Add elements to the model: variables, constraints, and objective 

- We can add variables to the model with the method ```model.addVar(lb=0.0, ub=float('inf'), vtype=GRB.CONTINUOUS, name="")```.
- You should specify the lower and upper bounds as well as domain using the arguments ```lb```, ```ub```, and ```vtype```, respectively.

<b>Note that the default lower bound is 0, and the default variable domain is continuous!<b>

In [31]:
# Note that these two variables have the same bounds and domain
x_1 = model.addVar(lb=0, ub=float('inf'), vtype=GRB.CONTINUOUS, name="x_1")
x_2 = model.addVar(name="x_2")

- Generally, we add constraints with the ```model.addConstr(lhs,direction,rhs,name="")``` method
- You can specify the expression of the left-hand side (```lhs```), ```direction``` ($>=$, $=$ or $<=$), and right-hand side (```rhs```) of the constraint as separate arguments. In the ```GRB```module, you can find the three signs ```GRB.GREATER_EQUAL```, ```GRB.EQUAL```, and ```GRB.LESS_EQUAL```.
- In this case, the constraints are linear so we can use the ```model.addLConstr(constr, name="")``` method.

In [32]:
constraint_1 = model.addLConstr(0.6*x_1 + 0.2*x_2, GRB.GREATER_EQUAL, 60, name='constraint_1')
constraint_2 = model.addLConstr(0.4*x_1 + 0.8*x_2, GRB.GREATER_EQUAL, 100, name='constraint_2')

<b>Note that it is important to store the variables and constraints in a meaningful way so you can easily access the values of the primal and dual variables after solving.<b>

- We define the objective function with the method ```model.setObjective(expr, sense=None)```. 
- You should specify the expression of the objective function **and the sense** of the optimization model, i.e. minimizing or maximizing. In the ```GRB``` module, you can find the two sense arguments ```GRB.MINIMIZE``` and ```GRB.MAXIMIZE```. 

In [33]:
model.setObjective(30*x_1 + 20*x_2, GRB.MINIMIZE)

- Now, we can solve the optimization problem with the method ```model.optimize```.

In [34]:
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 22.4.0 22E261)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x20d42a0c
Coefficient statistics:
  Matrix range     [2e-01, 8e-01]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 1e+02]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.600000e+02   0.000000e+00      0s
       2    3.9000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.900000000e+03


- We can check whether the problem was solved to optimality with ```model.status```.
- If so, we retrieve optimal objective function with ```model.ObjVal``` 
- and optimal primal and dual variable values with ```var.x``` and ```constr.Pi```, respectively.

In [35]:
if model.status == GRB.OPTIMAL:
    optimal_objective = model.ObjVal
    optimal_x_1 = x_1.x
    optimal_x_2 = x_2.x
    optimal_dual_1 = constraint_1.Pi
    optimal_dual_2 = constraint_2.Pi
    print(f"optimal objective: {optimal_objective}")
    print(f"optimal value of {x_1.VarName}: {optimal_x_1}")
    print(f"optimal value of {x_2.VarName}: {optimal_x_2}")
    print(f"optimal value of dual for {constraint_1.constrName}: {optimal_dual_1}")
    print(f"optimal value of dual for {constraint_2.constrName}: {optimal_dual_2}")
else:
    print(f"optimization of {model.ModelName} was not successful")

optimal objective: 3900.0
optimal value of x_1: 70.0
optimal value of x_2: 90.0
optimal value of dual for constraint_1: 40.0
optimal value of dual for constraint_2: 15.0


These steps are summarized (using another example) in this short video: 

[![Watch on YouTube](https://img.youtube.com/vi/7sMhvLn02P8/hqdefault.jpg)](https://www.youtube.com/watch?v=7sMhvLn02P8&list=PLaxOs-8sLebsGEsuo1FEmpyM1LFrzmQBN&index=2)

### Task: Economic Dispatch (ED) Problem

Now, let's solve the ED problem (as week in Week 3):

$$
  \begin{align}
      \min_{x_i} \quad &\sum_{i=1}^{G} c_i x_i \\
      \textrm{s.t.} \quad &0 \leq x_i \leq \overline{P}_i \Delta t \quad \forall i \in \{1,...,G\} \\
      & \sum_{i=1}^{3} x_i = \sum_{j=1}^{D} L_j \\
  \end{align}
$$

with:
- $G$ number of online generators
- $D$ number of inflexible loads
- $\Delta t$: time steps (in h)
- $x_i$: dispatch/setpoint of generator $i=1,...,G$ (in MWh)
- $c_i$: variable production cost of generator $i=1,...,G$ (in DKK/MWh)
- $\overline{P}_i$: max. capacity of generator $i=1,...,G$ (in MW)
- $L_j$: inflexible demand of load $j=1,...,D$ (in MWh)

**Note that the parameter $\Delta t$ is used to convert power in MW into energy in MWh. Since the time steps in the classic ED problem are 1 hour, it is often omitted.** 

We provide the following input data:

In [36]:
# Define ranges and indexes
N_GENERATORS = 3 #number of generators
N_LOADS = 1 #number of inflexible loads
time_step = 1 #time step in hours (Delta_t)
GENERATORS = range(3) #range of generators
LOADS = range(1) #range of inflexible Loads

# Set values of input parameters
generator_cost = [70,15,150] # Variable generators costs (c_i)
generator_capacity = [150,150,150] # Generators capacity (\Overline{P}_i)
load_capacity = [200] # Loads capacity (L_j)

- In the same way as in step 2, please initialize and solve the problem using ```gurobipy```.

In [37]:
#create and name a new gurobi model


In [38]:
#create decision variables


In [39]:
#add constraints


In [40]:
#add objective function


In [41]:
#solve optimization problem


In [42]:
#check status and print results


## Section 3: Standardized (matrix) formulation

Linear problems can be expressed in a more general way by defining the inputs before-hand as vectors and mnatrices, and making the rest of the code more general. <br>

Here is a general formulation of the simple optimization problem introduce in Section 2:

In [43]:
# Set values of input parameters and define decision variables names
VARIABLES = ['x1', 'x2'] # list of names of variables
objective_coeff = {'x1': 30, 'x2': 20} # Coefficients in objective function
constraints_coeff = {'x1': [0.6, 0.4], 'x2': [0.2, 0.8]} # Linear coefficients of constraints
constraints_rhs = [60, 100] # Right hand side coefficients of constraints
constraints_sense =  [GRB.GREATER_EQUAL, GRB.GREATER_EQUAL] # Direction of constraints

In [44]:
# create model
model = gp.Model("toy model")

In [45]:
# Add variables to the Gurobi model
variables = {v: model.addVar(lb=0, name=f'variable {v}') for v in VARIABLES}

In [46]:

# Set objective function and optimization direction of the Gurobi model
objective = gp.quicksum(objective_coeff[v] * variables[v] for v in VARIABLES)         
model.setObjective(objective, GRB.MINIMIZE)

In [47]:
# Add constraints to the Gurobi model
n_constraints = len(constraints_rhs)
constraints = [
                model.addLConstr(
                        gp.quicksum(constraints_coeff[v][i] * variables[v] for v in VARIABLES),
                        constraints_sense[i],
                        constraints_rhs[i],
                        name=f'constraint {i}'
                ) for i in range(n_constraints)
]


In [48]:
# Optimize the Gurobi model

model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 22.4.0 22E261)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x20d42a0c
Coefficient statistics:
  Matrix range     [2e-01, 8e-01]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 1e+02]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.600000e+02   0.000000e+00      0s
       2    3.9000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.900000000e+03


In [49]:
# Check if the optimization was successful and print solutions
if model.status == GRB.OPTIMAL:
    optimal_objective = model.ObjVal
    optimal_variables = {v.VarName: v.x for v in model.getVars()}
    optimal_duals = {c.ConstrName: c.Pi for c in model.getConstrs()}
    print(f"optimization of {model.ModelName} was successful")
    print('--------------------- RESULTS -----------------------')
    print('optimal objective:', optimal_objective)
    for v in model.getVars():
        print(f'optimal value of {v.VarName}:', v.x)
    for c in model.getConstrs():
        print(f'dual variable of {c.ConstrName}:', c.Pi)
else:
    print(f"optimization of {model.ModelName} was not successful")

optimization of toy model was successful
--------------------- RESULTS -----------------------
optimal objective: 3900.0
optimal value of variable x1: 70.0
optimal value of variable x2: 90.0
dual variable of constraint 0: 40.0
dual variable of constraint 1: 15.0


### Task: Solving ED problem

Now, use the general formulation to solve the ED problem (see section 2):

- We can then define an instance of this ED model and solve it:

In [50]:
# define input data in standard form

In [51]:
# create model and solve model

In [52]:
# check status and print results

## Section 4: Introduction to Object-oriented programming (OOP)?

### What is OOP?

- OOP is a very powerful tool to structure large optimization problems. 
- In this section, key concepts within OOP are introduced and in the next section, they are applied to the example problem from section 2.

### Classes 
OOP is all about classes. We'll use the class ```Dog``` (below) as a basis to discuss key concepts.

In [53]:
class Dog:

    def __init__(self, breed: str, age: int):
        self.breed = breed
        self.age = age 
    
    def bark(self):
        if self.breed == 'Bloodhound':
            print("WOOF WOOF")
        elif self.breed == "Chihuahua":
            print("woof woof")
        else: 
            raise NotImplementedError("I don't know the bark of this dog")

### Instance
We can create an object which is an instance of the class by providing the arguments ```breed``` and ```age```.

In [54]:
pluto = Dog('Bloodhound', 94)

### ```__init__``` method and attributes
When we created the instance ```pluto```, the ```self.__init__``` method was automatically called <br> 
and the two attributes ```self.breed``` and ```self.age``` were set. Here, ```self```refers to the instance. <br> 
We can access attributes outside of the class with ```instance.attribute```.

In [55]:
print(pluto.breed)
print(pluto.age)

Bloodhound
94


### Methods
Functions defined inside the class are called methods and these can be performed on instances of the class.<br>
The methods often use (or alter) attributes like a dog's bark depends on its breed.

In [56]:
print("Pluto barks: ")
pluto.bark()
harajuku = Dog("Chihuahua", 23)
print("Harajuku barks:")
harajuku.bark()

Pluto barks: 
WOOF WOOF
Harajuku barks:
woof woof


### Inheritance 

One class (let's call it class 1) can "extend" another class (class 2), which means it inherits <br>
the attributes and methods of class 2. Quite fittingly, class 2 is refered to as the parent class <br>
and class 1, the child class. The class definition looks like this: ```class Child(Parent):```. <br>
We continue the dog example below. 

In [57]:
class Chihuahua(Dog):

    def __init__(self, age: int, shake: str):
        self.breed = "Chihuahua"
        self.age = age
        self.shake = shake

In [58]:
tinkerbell = Chihuahua(14, 'strong')
tinkerbell.bark()

woof woof


- Notice how we can use the method ```Dog.bark()``` as it is defined in the parent class ```Dog```,
- and how we introduced a new attribute ```shake``` which is specific to Chihuahuas. 

### Example: Linear optimizatin problem with OOP

We define below a general class of Linear Optimization Problems and methods to initialize, solve and display its results. 

**Admittedly, it is a bit over the top to use OOP for the example problems above. However,<br> 
in the coming exercises/assignments, OOP will be a big help, in particular for solving multiple instances of the same optimization model and running numerical experiements in a structured way.**

- Firstly, we introduce a small class named ```Expando``` which allows for instance attributes to have attributes. (It will make sense later :))

In [59]:
class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass

- Then, we define an ```InputData``` class which holds the necessary data for the optimization problem. 
- Therefore, it has attributes like ```self.VARIABLES```, ```self.objective_coeff```, ```self.constraints_coeff```, etc.

In [65]:
class LP_InputData:

    def __init__(
        self, 
        VARIABLES: list[str],
        CONSTRAINTS: list[str],
        objective_coeff: dict[str, float],               # Coefficients in objective function
        constraints_coeff: dict[str, dict[str,float]],    # Linear coefficients of constraints
        constraints_rhs: dict[str, float],                # Right hand side coefficients of constraints
        constraints_sense: dict[str, int],              # Direction of constraints
        objective_sense: int,                           # Direction of op2timization
        model_name: str                                 # Name of model
    ):
        self.VARIABLES = VARIABLES
        self.CONSTRAINTS = CONSTRAINTS
        self.objective_coeff = objective_coeff
        self.constraints_coeff = constraints_coeff
        self.constraints_rhs = constraints_rhs
        self.constraints_sense = constraints_sense
        self.objective_sense = objective_sense
        self.model_name = model_name


- Now, we can define the class ```LP_OptimizationProblem```, which takes an instance of the InputData class as the only argument and stores it as ```self.data```.
- It has methods to build and solve the problem as well as save and display results. 

In [66]:
class LP_OptimizationProblem():

    def __init__(self, input_data: LP_InputData): # initialize class
        self.data = input_data # define data attributes
        self.results = Expando() # define results attributes
        self._build_model() # build gurobi model
    
    def _build_variables(self):
        self.variables = {v: self.model.addVar(lb=0, name=f'{v}') for v in self.data.VARIABLES}
    
    def _build_constraints(self):
        self.constraints = {c:
                self.model.addLConstr(
                        gp.quicksum(self.data.constraints_coeff[c][v] * self.variables[v] for v in self.data.VARIABLES),
                        self.data.constraints_sense[c],
                        self.data.constraints_rhs[c],
                        name = f'{c}'
                ) for c in self.data.CONSTRAINTS
        }

    def _build_objective_function(self):
        objective = gp.quicksum(self.data.objective_coeff[v] * self.variables[v] for v in self.data.VARIABLES)
        self.model.setObjective(objective, self.data.objective_sense)

    def _build_model(self):
        self.model = gp.Model(name=self.data.model_name)
        self._build_variables()
        self._build_objective_function()
        self._build_constraints()
        self.model.update()
    
    def _save_results(self):
        self.results.objective_value = self.model.ObjVal
        self.results.variables = {v.VarName:v.x for v in self.model.getVars()}
        self.results.optimal_duals = {f'{c.ConstrName}':c.Pi for c in self.model.getConstrs()}

    def run(self):
        self.model.optimize()
        if self.model.status == GRB.OPTIMAL:
            self._save_results()
        else:
            print(f"optimization of {model.ModelName} was not successful")
    
    def display_results(self):
        print()
        print("-------------------   RESULTS  -------------------")
        print("Optimal objective:", self.results.objective_value)
        for key, value in self.results.variables.items():
                print(f'Optimal value of {key}:', value)
        for key, value in self.results.optimal_duals.items():
                print(f'Dual variable of {key}:', value)

- Notice how ```self.results = Expando()``` allows us to save different results in the ```self.results``` attribute, e.g., ```self.results.objective_value```.

#### Toy Model and ED problems

- Below is what corresponds to the ```main``` function where we create instances of the classes and use their methods for the toy model and ED models introduced in Section 2. 

In [68]:
# this corresponds to the main function 

input_data = LP_InputData(
        VARIABLES = ['x1', 'x2'],
        CONSTRAINTS = ['c1','c2'],
        objective_coeff = {'x1': 30, 'x2': 20},
        constraints_coeff = {'c1': {'x1':0.6,'x2':0.2},'c2': {'x1':0.4,'x2':0.8}},
        constraints_rhs = {'c1':60, 'c2':100},
        constraints_sense =  {'c1':GRB.GREATER_EQUAL, 'c2':GRB.GREATER_EQUAL},
        objective_sense = GRB.MINIMIZE,
        model_name = "toy problem")

In [69]:
# create and solve the model

model = LP_OptimizationProblem(input_data)
    
model.run()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 22.4.0 22E261)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x20d42a0c
Coefficient statistics:
  Matrix range     [2e-01, 8e-01]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 1e+02]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.600000e+02   0.000000e+00      0s
       2    3.9000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.900000000e+03


In [74]:
print(f'------------------- {model.data.model_name}  -------------------')
model.display_results()
print(f'--------------------------------------------------')

------------------- toy problem  -------------------

-------------------   RESULTS  -------------------
Optimal objective: 3900.0
Optimal value of x1: 70.0
Optimal value of x2: 90.0
Dual variable of c1: 40.0
Dual variable of c2: 15.0
--------------------------------------------------
